setup

In [66]:
import pandas as pd
from datetime import datetime
start_time = datetime.now()


set up sorter

In [67]:
def sort_Key(word):

    pāli_alphabet = [
        "√", "a", "ā", "i", "ī", "u", "ū", "e", "o", "k", "kh", "g", "gh", "ṅ",
        "c", "ch", "j", "jh", "ñ", "ṭ", "ṭh", "ḍ", "ḍḥ", "ṇ", "t", "th", "d",
        "dh", "n", "p", "ph", "b", "bh", "m", "y", "r", "l", "s", "v", "h",
        "ḷ", "ṃ", " ", "1", "2", "3", "4", "5", "6", "7", "8", "9", "0"
    ]

    #comp iteration condition
    dl = [i for i in pāli_alphabet if len(i) > 1]

    for i in dl:
        word = word.replace(i, '/{}'.format(i))

    wordVe = []

    k = -3

    for j in range(len(word)):
        if word[j] == '/':
            k = j
            wordVe.append(word[j + 1:j + 3])
        if j > k + 2:
            wordVe.append(word[j])

    word = wordVe

    pāli_alphabet_string = '-'.join(pāli_alphabet)
    return [pāli_alphabet_string.find('-' + x + '-') for x in wordVe]
sort_Key

<function __main__.sort_Key(word)>

create roots_db

In [68]:
roots_df = pd.read_csv("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/roots.csv", sep="\t", dtype=str)
roots_df.fillna("", inplace=True)
roots_df.replace("\.0$", "", inplace=True, regex=True)
roots_df = roots_df[roots_df["Count"] != "0"] # remove roots with no examples
# roots_df.sort_values(by = ["Root"], inplace=True, ignore_index=True, key=lambda x: x.map(sortKey)) #sort
roots_df["Dhātupātha"] = roots_df["Dhātupātha"].str.replace("-", "")
roots_df["Kaccāyana Dhātu Mañjūsā"] = roots_df["Kaccāyana Dhātu Mañjūsā"].str.replace("-", "")

# row count
roots_df_count = roots_df.shape[0]

setup dpd_df wi

In [69]:
dpd_df = pd.read_csv("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/dpd-full.csv", sep="\t", dtype=str)
dpd_df.fillna("", inplace=True)

dpd_df.loc[dpd_df["Meaning IN CONTEXT"] == "", "Meaning IN CONTEXT"] = dpd_df["Buddhadatta"] + "*"

extract families from dpd-full.csv

In [70]:
test1 = dpd_df["Family"] != ""
test2 = dpd_df["Metadata"] == ""
filter = test1 & test2
root_families_df = dpd_df.loc[filter, ["Pāli Root", "Grp", "Root Meaning", "Family"]]
root_families_df = root_families_df.drop_duplicates(subset=["Pāli Root", "Grp", "Root Meaning", "Family"])
root_families_df.sort_values(["Pāli Root", "Grp", "Root Meaning", "Family"], ascending = (True, True, True, True), inplace=True)
# root_families_df.sort_values(by = ["Family"], inplace=True, ignore_index=True, key=lambda x: x.map(sortKey)) #sort
root_families_df = root_families_df.reset_index(drop=True)

# row count
root_families_df_count = root_families_df.shape[0]

generate html for each root subfamily

In [72]:
# print(root_families_df.columns)

for row in range(root_families_df_count):
	root = root_families_df.loc[row, "Pāli Root"]
	root_group = root_families_df.loc[row, "Grp"]
	root_meaning = root_families_df.loc[row, "Root Meaning"]
	subfamily = root_families_df.loc[row, "Family"]

	# dpd_df.columns
	test1 = dpd_df["Pāli Root"] == root
	test2 = dpd_df["Grp"] == root_group
	test3 = dpd_df["Root Meaning"] == root_meaning
	test4 = dpd_df["Family"] == subfamily
	filter = test1 & test2 & test3 & test4
	subfamily_df = dpd_df.loc[filter, ["Pāli1", "POS", "Meaning IN CONTEXT"]]

	subfamily_df_length = subfamily_df.shape[0]

	with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/subfamily html/{root} {root_group} {root_meaning} {subfamily}.html", "w") as output_file:
			
		output_file.write(f"""<table class= "table3"><tbody>""")
		
		for row_sf in range(subfamily_df_length):
			sf_pali = subfamily_df.iloc[row_sf, 0]
			sf_pos = subfamily_df.iloc[row_sf, 1]
			sf_english = subfamily_df.iloc[row_sf, 2]
			# print(f"{row_sf}\t{sf_pali}\t({sf_pos})  \t{sf_english}")


			output_file.write(f"<tr><th>{sf_pali}</th>")
			output_file.write(f"<td>{sf_pos}</td>")
			output_file.write(f"<td>{sf_english}</td></tr>")
		
		output_file.write(f"""</tbody></table>""")


timer

In [74]:
end_time = datetime.now()
print('{} total'.format(end_time - start_time))

0:00:28.462811 total
